# Lab2
## Overview
การสร้างแบบจำลองของหุ่นยนต์ แสดงผลหุ่นยนต์ผ่าน rviz2 และหาความสัมพันธ์ของจลนศาสตร์ 3-DOF manipulator's โดยใช้ ROS2 framework<br>
โดยประกอบไปด้วย 1. การออกแบบแขนกล 2. การสร้าง URDF และแสดงผล 3. Controller
# 1.การออกแบบแขนกล 
การสร้างโมเดลแขนกลโดยดราใช้ Solidwork ในการออกแบบ แขนกล 3 DOF ซึ่งเป็น Revolute Joint ทั้งหมด

<p align="center">
<img src="pic/pic1.png" width="40%">
</p>

<p align="center">
ภาพที่ 1 ภาพโมเดลแขนกล
</p>

กำหนด Coordinate Frame ของแต่ละชิ้นส่วนดังภาพ
<p align="center">
<img src="pic/pic2.png" width="40%">
</p>

<p align="center">
ภาพที่ 2 ภาพตำแหน่ง Coordinate Frame ของแต่ชิ้นส่วนของแขนกล
</p>

# 2.การสร้าง URDF และแสดงผล 
เราใช้ package robot_state_publisher ในการอ่าน link ในไฟล์ urdf และนำไปแสดงผลใน rviz2 ซึ่งอยู่ใน __(จะอยู่ใน code display.launch.py line 31-37 (robot_state_publisher)__
และใช้ package joint_state_publisher ในการส่งค่า sensor_msgs/JointState ซึ่งเป็นค่าการวางตำแหน่งของแต่ละ joint 

```urdf
    <joint name="joint_1" type="revolute">
        <parent link="link_0" />
        <child link="link_1" />
        <origin xyz="0 0 0.7" rpy="0 0 0" />
        <axis xyz="0 0 1"/>
        <limit lower="-3.14" upper="3.14" effort="0.5" velocity="0.5"/>
    </joint>
```
โดย code นี้เป็นตัวอย่างของการกำหนดตำแหน่งระหว่างเฟรมของ 2 ชิ้นส่วน โดยเราได้ไฟล์ launch สำหรับการแสดงผลแขนกลและมี gui สำหรับปรับองศาการหมุนของแต่ละ Frame นอกจากนี้ยังมีแบบที่ไม่มี gui สำหรับ kinematic package

<p align="center">
<img src="pic/pic3.png" width="80%">
</p>

<p align="center">
ภาพที่ 3 การแสดงผลใน rviz
</p>

# 3.Controller
เราสร้าง custom service ขึ้นมา 2 service คือ SetFK.srv และ SolveIK.srv โดย SetFK.srv มีหน้าที่ในการรับค่า องศาของ Frame และส่งค่าตำแหน่งของ End_Effector เมื่อเทียบ Frame 0 ส่วน SolveIK.srv รับค่าตำแหน่ง End_Effector เทียบ Frame 0 และ  Joint Config และส่งค่า องศาของแต่ละ Frame กลับมา<br>
โดย Controller ของเรานั้นมี Server 1.SetJoint ทำ Forward Kinematics ของแขนกล 2.solve_ik ทำ Inverse Kinematics และมี publisher สำหรับ publish Joint State เพื่อส่งให้ robot_state_publisher

3.1 forward kinematics
โดยแขนกลนั้นมี dh parameter ดังนี้
<p align="center">
<img src="pic/pic4.png" width="40%">
</p>

<p align="center">
ภาพที่ 4 ตาราง dh parameter
</p>

โดยในการทำ forward kinematics ของแขนกลนั้น จะใช้คำนวณตามสูตรดังนี้คือการคำนวณ homogenous transformation matrix ของแต่ละ frame โดยเป็นการคำนวณที่ละ frame จาก frame 0 ไปจนถึง frame 3 โดยแต่ล่ะ frame จะมีการคำนวณตามสูตรดังนี้
<p align="center">
<img src="pic/pic5.png" width="40%">
</p>
<p align="center">
ภาพที่ 5 สูตรคำนวณ homogenous transformation matrix ของแต่ละ frame จาก dh parameter
</p>

โดยแต่ล่ะ frame จะต้องมีการคูณ matrix อีกตัวที่เป็นองศาของ Frame นั้นๆ โดยจะมีการคำนวณตามสูตรดังนี้
<p align="center">
<img src="pic/pic6.png" width="40%">
</p>

<p align="center">
ภาพที่ 6 สูตรคำนวณ homogenous transformation matrix ของ frame นั้นๆ
</p>
หลังจากการคำนวณ homogenous transformation matrix ของแต่ละ frame จะได้ matrix ของ frame 3 แล้ว จากนั้นจะทำการคำนวณตำแหน่งของ End_Effector โดยจะใช้สูตรดังนี้
<p align="center">
<img src="pic/pic7.png" width="40%">
</p>

<p align="center">
ภาพที่ 7 สูตรคำนวณตำแหน่งของ End_Effector
</p>
จากนนั้นทำการส่งค่าตำแหน่งของ End_Effector กลับไปให้ client และนำค่าไปเก็บไว้ในคลาสเพื่อใช้สำหรับส่งค่า publish Joint State เพื่อส่งให้ robot_state_publisher



3.2 inverse kinematics
โดยในการทำ inverse kinematics ของแขนกลนั้น จะเป็นใช้วิธีการเรขาคณิตในการหาค่าตำแหน่งของ Joint โดยจะใช้สูตรดังนี้
<p align="center">
<img src="pic/pic8.png" width="20%">
</p>

<p align="center">
ภาพที่ 8 การระบุตำแหน่งของ End_Effector ใน plane X0Y0
</p>
และได้เป็นเมทริกซ์ดังนี้ และสามารถคำนวณหา sin1 และ cos1 ได้จากสูตรดังนี้
<p align="center">
<img src="pic/pic9.png" width="20%">
</p>

<p align="center">
ภาพที่ 9 สมการของการระบุองศาของ Joint 1 (a มีค่าเท่ากับ 0.1)
</p>

<p align="center">
<img src="pic/pic10.png" width="20%">
</p>

<p align="center">
ภาพที่ 10 การระบุตำแหน่งของ End_Effector ใน plane Z0R0 โดยแกน R0 คือ แกน x ที่ถูกหมุนออกโดยแกน Z ด้วยมุมarctan(10/205) เพื่อระนาบของตำแหน่ง end effector นั้นเป็นเส้นตรง
โดยเราจะได้รว่าแขนกลนั้นเป็น revolute 2 joint ซึ่งสามารถแก้โดยใช้ RRIK แก้ได้ดังนี้
</p>
<p align="center">
<img src="pic/pic11.png" width="40%">
</p>

<p align="center">
ภาพที่ 11 สมการระบุองศาของ Joint 2 และ Joint 3
โดน l0 คือความยาวของ link 0 ในแกน x1 มีค่าเท่ากับ 0.1, l1 คือความยาวของ link 1 ในแกน x2 มีค่าเท่ากับ 1, l2 คือความยาวของ link 2 ในแกน x3 มีค่าเท่ากับ 1.15 และ h คือความสูงของ End_Effector มีค่าเท่ากับ 0.7

หลังจากนั้นเราทำการนำสมการที่ได้นั้นไปใส่ในโปรแกรม โดยจะรับค่าตำแหน่งของ End_Effector จาก client และนำไปคำนวณค่าของ Joint 1 Joint 2  และ Joint 3 และนำค่าไปเก็บไว้ในคลาสเพื่อใช้สำหรับส่งค่า publish Joint State เพื่อส่งให้ robot_state_publisher
<p align="center">
<img src="pic/pic12.png" width="80%">
</p>

<p align="center">
ภาพที่ 12 การทำงาน service ของ inverse kinematics
</p>